In [1]:
# Upload your SMSSpamCollection file
from google.colab import files
uploaded = files.upload()

# Load dataset
import pandas as pd

df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])
print("\nFirst 5 rows:")
df.head()


Saving SMSSpamCollection to SMSSpamCollection

First 5 rows:


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# After restart, import
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy
spacy.cli.download("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
# Basic info
print("\nShape:", df.shape)
print("\nColumns:", df.columns)
print("\nData Types:\n", df.dtypes)

# Check missing values
print("\nMissing Values:\n", df.isnull().sum())

# Remove duplicates
print("\nDuplicate Rows:", df.duplicated().sum())
df = df.drop_duplicates()
print("Shape after removing duplicates:", df.shape)

# Check class distribution
print("\nClass Distribution:\n", df['label'].value_counts())




Shape: (5572, 2)

Columns: Index(['label', 'message'], dtype='object')

Data Types:
 label      object
message    object
dtype: object

Missing Values:
 label      0
message    0
dtype: int64

Duplicate Rows: 403
Shape after removing duplicates: (5169, 2)

Class Distribution:
 label
ham     4516
spam     653
Name: count, dtype: int64


In [3]:
# Step 1: Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

# Step 2: If duplicates exist, remove them
if duplicate_rows > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape of dataset: {df.shape}")
else:
    print("No duplicate rows found. No removal needed.")

# Step 3: View dataset after removing duplicates (optional)
df.head()


Number of duplicate rows: 0
No duplicate rows found. No removal needed.


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Check the distribution again
label_distribution = df['label'].value_counts()
print("Class distribution before balancing:")
print(label_distribution)

# Calculate imbalance ratio
ham_count = label_distribution['ham']
spam_count = label_distribution['spam']

print(f"\nHam Count: {ham_count}")
print(f"Spam Count: {spam_count}")

imbalance_ratio = ham_count / spam_count
print(f"Imbalance Ratio (ham/spam): {imbalance_ratio:.2f}")


Class distribution before balancing:
label
ham     4516
spam     653
Name: count, dtype: int64

Ham Count: 4516
Spam Count: 653
Imbalance Ratio (ham/spam): 6.92


In [5]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters, numbers
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Apply preprocessing
df_balanced['clean_message'] = df_balanced['message'].apply(preprocess)


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [6]:
# Step 1: Install and Import Libraries (if not already done)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Step 2: Upload and Load the Dataset
from google.colab import files
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

# Step 3: Preprocess Text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()                             # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)             # Remove punctuation/numbers
    tokens = word_tokenize(text)                     # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]      # Lemmatization
    return ' '.join(tokens)

# Apply preprocessing
df['clean_message'] = df['message'].apply(preprocess)

# Step 4: Find Most Frequent Words
all_words = ' '.join(df['clean_message']).split()

# Count word frequencies
word_freq = Counter(all_words)

# Get top 10 most common words
most_common_words = word_freq.most_common(10)

print("\nTop 10 Most Frequently Occurring Words in the Dataset:")
for word, freq in most_common_words:
    print(f"{word}: {freq} times")

# Step 5: (Optional) Visualize Top Words
words, counts = zip(*most_common_words)

plt.figure(figsize=(10,5))
sns.barplot(x=list(words), y=list(counts), palette="viridis")
plt.title('Top 10 Most Frequent Words')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Saving SMSSpamCollection to SMSSpamCollection (1)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [7]:
# Step 1: Install and Import Libraries (if not already done)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Step 2: Upload and Load the Dataset
from google.colab import files
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

# Step 3: Preprocess Text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()                             # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)             # Remove punctuation/numbers
    tokens = word_tokenize(text)                     # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]      # Lemmatization
    return ' '.join(tokens)

# Apply preprocessing
df['clean_message'] = df['message'].apply(preprocess)

# Step 4: Filter Only 'Spam' Messages
spam_messages = df[df['label'] == 'spam']['clean_message']

# Combine all spam words
spam_words = ' '.join(spam_messages).split()

# Count spam word frequencies
spam_word_freq = Counter(spam_words)

# Get top 10 most common spam words
most_common_spam_words = spam_word_freq.most_common(10)

print("\nTop 10 Most Common Words in 'Spam' Messages:")
for word, freq in most_common_spam_words:
    print(f"{word}: {freq} times")

# Step 5: (Optional) Visualize Top Spam Words
words, counts = zip(*most_common_spam_words)

plt.figure(figsize=(10,5))
sns.barplot(x=list(words), y=list(counts), palette="mako")
plt.title('Top 10 Most Frequent Words in Spam Messages')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Saving SMSSpamCollection to SMSSpamCollection (2)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
